In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment
from azure.identity import InteractiveBrowserCredential

In [ ]:
#enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

In [ ]:
#create an online endpoint
endpoint = ManagedOnlineEndpoint(name='my-online-endpoint',
            description='this is a sample online endpoint',
            auth_mode='key',
            tags={'foo': 'bar'})
                     
ml_client.begin_create_or_update(endpoint)

In [ ]:
#create an online deployment
model = Model(local_path='./model-1/model/sklearn_regression_model.pkl')
env = Environment(conda_file='./model-1/environment/conda.yml', image='mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1')

blue_deployment = ManagedOnlineDeployment(
    name='blue',
    endpoint_name='my-online-endpoint',
    model=model,
    environment=env,
    code_local_path="./model-1/onlinescoring",
    scoring_script="score.py",
    instance_type='Standard_F2s_v2',
    instance_count=1)

ml_client.begin_create_or_update(blue_deployment)

In [ ]:
#test the blue deployment with some sample data
ml_client.online_endpoints.invoke(name='my-online-endpoint', request_file='./model-1/sample-request.json')

In [ ]:
#scale the deployment
blue_deployment = ml_client.online_deployments.get(name='blue', endpoint_name='my-online-endpoint') #redundant step, added just to show the get feature
blue_deployment.instance_count = 2
ml_client.online_deployments.create_or_update(blue_deployment)

In [ ]:
#create a second deployment
model2 = Model(local_path='./model-2/model/sklearn_regression_model.pkl')
env2 = Environment(conda_file='./model-2/environment/conda.yml', image='mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1')

green_deployment = ManagedOnlineDeployment(
    name='green',
    endpoint_name='my-online-endpoint',
    model=model2,
    environment=env2,
    code_local_path="./model-2/onlinescoring",
    scoring_script="score.py",
    instance_type='Standard_F2s_v2',
    instance_count=1)

ml_client.begin_create_or_update(green_deployment)

In [ ]:
# test the green deployment

ml_client.online_endpoints.invoke(name='my-online-endpoint', deployment_name='green', request_file='./model-2/sample-request.json')

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name='my-online-endpoint')
#existing traffic details
print(endpoint.traffic)
#Get the scoring URI
print(endpoint.scoring_uri)

In [ ]:
#Update the traffic to divert 10% of traffic to the green deployment
endpoint.traffic = {'blue': 90, 'green': 10}
ml_client.begin_create_or_update(endpoint)

In [ ]:
#Get the logs for the green deployment and verify as needed
ml_client.online_deployments.get_logs(name='green', endpoint_name='my-online-endpoint')

In [ ]:
#now divert 100% of the traffic to the green deployment
endpoint.traffic = {'blue': 0, 'green': 100}
ml_client.begin_create_or_update(endpoint)

In [ ]:
#delete the blue deployment
mlclient.online_deployments.delete(name='blue', endpoint_name='my-online-endpoint')